### Seldon V2 Batch Examples

In [1]:
import os
os.environ["NAMESPACE"] = "seldon"

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n ${NAMESPACE} -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.19.255.12'

## Deploy Models and Pipelines

In [3]:
!cat models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn
  memory: 100Ki


In [4]:
!cat pipelines/iris.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: iris-pipeline
spec:
  steps:
    - name: iris
  output:
    steps:
    - iris


In [5]:
!cat models/tfsimple1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki


In [6]:
!cat pipelines/tfsimple.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimple
spec:
  steps:
    - name: tfsimple1
  output:
    steps:
    - tfsimple1


In [7]:
!kubectl apply -f models/sklearn-iris-gs.yaml -n ${NAMESPACE}
!kubectl apply -f pipelines/iris.yaml -n ${NAMESPACE}

!kubectl apply -f models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl apply -f pipelines/tfsimple.yaml -n ${NAMESPACE}

model.mlops.seldon.io/iris unchanged
pipeline.mlops.seldon.io/iris-pipeline unchanged
model.mlops.seldon.io/tfsimple1 unchanged
pipeline.mlops.seldon.io/tfsimple unchanged


In [8]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}
!kubectl wait --for condition=ready --timeout=300s pipelines --all -n ${NAMESPACE}

!kubectl wait --for condition=ready --timeout=300s model tfsimple1 -n ${NAMESPACE}
!kubectl wait --for condition=ready --timeout=300s pipelines tfsimple -n ${NAMESPACE}

model.mlops.seldon.io/iris condition met
model.mlops.seldon.io/tfsimple condition met
model.mlops.seldon.io/tfsimple1 condition met
pipeline.mlops.seldon.io/iris-pipeline condition met
pipeline.mlops.seldon.io/tfsimple condition met
pipeline.mlops.seldon.io/tfsimple-pipeline condition met
model.mlops.seldon.io/tfsimple1 condition met
pipeline.mlops.seldon.io/tfsimple condition met


## Test Predictions

In [9]:
!seldon model infer iris --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' | jq -M .

{
  "model_name": "iris_1",
  "model_version": "1",
  "id": "5b88fb3f-b467-4c36-b172-0ca81ecfab2e",
  "parameters": {
    "content_type": null,
    "headers": null
  },
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "INT64",
      "parameters": null,
      "data": [
        2
      ]
    }
  ]
}


In [10]:
!seldon pipeline infer iris-pipeline --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' |  jq -M .

{
  "model_name": "",
  "outputs": [
    {
      "data": [
        2
      ],
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "INT64"
    }
  ]
}


In [11]:
!seldon model infer tfsimple --inference-host ${MESH_IP}:80 \
  '{"outputs":[{"name":"OUTPUT0"}], "inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "model_name": "tfsimple_1",
  "model_version": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        1,
        16
      ],
      "data": [
        2,
        4,
        6,
        8,
        10,
        12,
        14,
        16,
        18,
        20,
        22,
        24,
        26,
        28,
        30,
        32
      ]
    }
  ]
}


In [12]:
!seldon pipeline infer tfsimple-pipeline --inference-host ${MESH_IP}:80 \
  '{"outputs":[{"name":"OUTPUT0"}], "inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "model_name": "",
  "outputs": [
    {
      "data": [
        2,
        4,
        6,
        8,
        10,
        12,
        14,
        16,
        18,
        20,
        22,
        24,
        26,
        28,
        30,
        32
      ],
      "name": "OUTPUT0",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    },
    {
      "data": [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0
      ],
      "name": "OUTPUT1",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    }
  ]
}


## MLServer Iris Batch Job

In [13]:
!cat batch-inputs/iris-input.txt | head -n 1 | jq .

{
  "inputs": [
    {
      "name": "predict",
      "data": [
        0.38606369295833043,
        0.006894049558299753,
        0.6104082981607108,
        0.3958954239450676
      ],
      "datatype": "FP64",
      "shape": [
        1,
        4
      ]
    }
  ]
}


In [14]:
%%bash
mlserver infer -u ${MESH_IP} -m iris -i batch-inputs/iris-input.txt -o /tmp/iris-output.txt --workers 5

2022-10-10 08:40:18,854 [mlserver] INFO - Using asyncio event-loop policy: uvloop
2022-10-10 08:40:18,857 [mlserver] INFO - Server url: 172.19.255.12
2022-10-10 08:40:18,857 [mlserver] INFO - input file path: batch-inputs/iris-input.txt
2022-10-10 08:40:18,857 [mlserver] INFO - output file path: /tmp/iris-output.txt
2022-10-10 08:40:19,066 [mlserver] INFO - Time taken: 0.21 seconds


In [15]:
%%bash
mlserver infer -u ${MESH_IP} -m iris-pipeline.pipeline -i batch-inputs/iris-input.txt -o /tmp/iris-pipeline-output.txt --workers 5

2022-10-10 08:40:20,849 [mlserver] INFO - Using asyncio event-loop policy: uvloop
2022-10-10 08:40:20,852 [mlserver] INFO - Server url: 172.19.255.12
2022-10-10 08:40:20,852 [mlserver] INFO - input file path: batch-inputs/iris-input.txt
2022-10-10 08:40:20,852 [mlserver] INFO - output file path: /tmp/iris-pipeline-output.txt
2022-10-10 08:40:21,307 [mlserver] INFO - Time taken: 0.45 seconds


In [16]:
!cat /tmp/iris-output.txt | head -n 1 | jq .

{
  "model_name": "iris_1",
  "model_version": "1",
  "id": "08359080-6fe2-4429-8c37-cc6276f4340b",
  "parameters": {
    "content_type": null,
    "headers": null,
    "batch_index": 0
  },
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "INT64",
      "parameters": null,
      "data": [
        0
      ]
    }
  ]
}


In [17]:
!cat /tmp/iris-pipeline-output.txt | head -n 1 | jq .

{
  "model_name": "",
  "outputs": [
    {
      "data": [
        0
      ],
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "INT64"
    }
  ],
  "parameters": {
    "batch_index": 0
  }
}


## Triton TFSimple Batch Job

In [18]:
!cat batch-inputs/tfsimple-input.txt | head -n 1 | jq .

{
  "inputs": [
    {
      "name": "INPUT0",
      "data": [
        75,
        39,
        9,
        44,
        32,
        97,
        99,
        40,
        13,
        27,
        25,
        36,
        18,
        77,
        62,
        60
      ],
      "datatype": "INT32",
      "shape": [
        1,
        16
      ]
    },
    {
      "name": "INPUT1",
      "data": [
        39,
        7,
        14,
        58,
        13,
        88,
        98,
        66,
        97,
        57,
        49,
        3,
        49,
        63,
        37,
        12
      ],
      "datatype": "INT32",
      "shape": [
        1,
        16
      ]
    }
  ]
}


In [19]:
%%bash
mlserver infer -u ${MESH_IP} -m tfsimple -i batch-inputs/tfsimple-input.txt -o /tmp/tfsimple-output.txt --workers 5

2022-10-10 08:40:23,543 [mlserver] INFO - Using asyncio event-loop policy: uvloop
2022-10-10 08:40:23,546 [mlserver] INFO - Server url: 172.19.255.12
2022-10-10 08:40:23,546 [mlserver] INFO - input file path: batch-inputs/tfsimple-input.txt
2022-10-10 08:40:23,546 [mlserver] INFO - output file path: /tmp/tfsimple-output.txt
2022-10-10 08:40:23,733 [mlserver] INFO - Time taken: 0.19 seconds


In [20]:
%%bash
mlserver infer -u ${MESH_IP} -m tfsimple-pipeline.pipeline -i batch-inputs/tfsimple-input.txt -o /tmp/tfsimple-pipeline-output.txt --workers 5

2022-10-10 08:40:25,397 [mlserver] INFO - Using asyncio event-loop policy: uvloop
2022-10-10 08:40:25,400 [mlserver] INFO - Server url: 172.19.255.12
2022-10-10 08:40:25,400 [mlserver] INFO - input file path: batch-inputs/tfsimple-input.txt
2022-10-10 08:40:25,400 [mlserver] INFO - output file path: /tmp/tfsimple-pipeline-output.txt
2022-10-10 08:40:25,752 [mlserver] INFO - Time taken: 0.35 seconds


In [21]:
!cat /tmp/tfsimple-output.txt | head -n 1 | jq .

{
  "id": "168eef39-0510-4729-b8df-3d0ec063df55",
  "model_name": "tfsimple_1",
  "model_version": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        1,
        16
      ],
      "parameters": {},
      "data": [
        114,
        46,
        23,
        102,
        45,
        185,
        197,
        106,
        110,
        84,
        74,
        39,
        67,
        140,
        99,
        72
      ]
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        1,
        16
      ],
      "parameters": {},
      "data": [
        36,
        32,
        -5,
        -14,
        19,
        9,
        1,
        -26,
        -84,
        -30,
        -24,
        33,
        -31,
        14,
        25,
        48
      ]
    }
  ],
  "parameters": {
    "batch_index": 0
  }
}


In [22]:
!cat /tmp/tfsimple-pipeline-output.txt | head -n 1 | jq .

{
  "model_name": "",
  "outputs": [
    {
      "data": [
        114,
        46,
        23,
        102,
        45,
        185,
        197,
        106,
        110,
        84,
        74,
        39,
        67,
        140,
        99,
        72
      ],
      "name": "OUTPUT0",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    },
    {
      "data": [
        36,
        32,
        -5,
        -14,
        19,
        9,
        1,
        -26,
        -84,
        -30,
        -24,
        33,
        -31,
        14,
        25,
        48
      ],
      "name": "OUTPUT1",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    }
  ],
  "parameters": {
    "batch_index": 0
  }
}


## Cleanup

In [23]:
!kubectl delete -f models/sklearn-iris-gs.yaml -n ${NAMESPACE}
!kubectl delete -f pipelines/iris.yaml -n ${NAMESPACE}

model.mlops.seldon.io "iris" deleted
pipeline.mlops.seldon.io "iris-pipeline" deleted


In [24]:
!kubectl delete -f models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl delete -f pipelines/tfsimple.yaml -n ${NAMESPACE}

model.mlops.seldon.io "tfsimple1" deleted
pipeline.mlops.seldon.io "tfsimple" deleted
